### Cleaning the Inspection Data:
The first part of this is cleaning the inspection data and creating a new csv file

In [7]:
# Import dependencies
from pprint import pprint
import pandas as pd 
import requests
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
from sklearn import datasets

#yelp_api_key = 

In [8]:
#import food inspection CSV file
inspection = pd.read_csv('inspection_5years.csv')
inspection.head(5)

,Unnamed: 0,AKA Name,Facility Type,Risk,Address,Zip,Year,Inspection Type,Results,Violations,Latitude,Longitude
0,0,RED MANGO,Restaurant,Risk 2 (Medium),1950 W POLK ST,60612.0,2022,License,Pass,NaN,41.871596,-87.675821
1,1649,WENDY'S,Restaurant,Risk 1 (High),4412 N PULASKI RD,60630.0,2022,Complaint Re-Inspection,Pass,NaN,41.961366,-87.727987
2,1663,GIORDANO'S PIZZA,Restaurant,Risk 1 (High),1401 W 18TH ST,60608.0,2022,Canvass,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.857765,-87.661385
3,1664,ALEX LA MICHOACANA,Restaurant,Risk 1 (High),5420 W FULLERTON AVE,60639.0,2022,Canvass Re-Inspection,Pass,NaN,41.924141,-87.761921
4,1667,KYO MATCHA,Restaurant,Risk 2 (Medium),2167 S CHINA PL,60616.0,2022,License,Pass,NaN,41.853556,-87.635198


In [9]:
#Change the format of the address
inspection = pd.DataFrame(inspection, columns = ['Address','AKA Name','Facility Type','Risk',
                                                'Zip', 'Year','Inspection Type','Results','Violations','Latitude','Longitude'])
inspection['Address'] = inspection['Address'].str.lower()

print(inspection)

                         Address                    AKA Name Facility Type  \
0                1950 w polk st                    RED MANGO    Restaurant   
1             4412 n pulaski rd                      WENDY'S    Restaurant   
2                1401 w 18th st             GIORDANO'S PIZZA    Restaurant   
3          5420 w fullerton ave           ALEX LA MICHOACANA    Restaurant   
4               2167 s china pl                   KYO MATCHA    Restaurant   
...                          ...                         ...           ...   
1624             1820 w 18th st   FIESTA TAMALERIA EL BARRIO    Restaurant   
1625  4212-4214 n milwaukee ave          LA PENA RESTAURANTE    Restaurant   
1626       1139 w bryn mawr ave         BEN'S NOODLES & RICE    Restaurant   
1627    8645 s stony island ave                      WENDY'S    Restaurant   
1628        2351 w augusta blvd    JOEY'S G'S MAC AND CHEESE    Restaurant   

                 Risk      Zip  Year          Inspection Type R

In [13]:
inspection = pd.DataFrame(inspection, columns = ['Address','AKA Name','Facility Type','Risk',
                                                'Zip', 'Year','Inspection Type','Results','Violations','Latitude','Longitude'])

inspection['Address'] = inspection['Address'].str.title()

print(inspection)

                         Address                    AKA Name Facility Type  \
0                1950 W Polk St                    RED MANGO    Restaurant   
1             4412 N Pulaski Rd                      WENDY'S    Restaurant   
2                1401 W 18Th St             GIORDANO'S PIZZA    Restaurant   
3          5420 W Fullerton Ave           ALEX LA MICHOACANA    Restaurant   
4               2167 S China Pl                   KYO MATCHA    Restaurant   
...                          ...                         ...           ...   
1624             1820 W 18Th St   FIESTA TAMALERIA EL BARRIO    Restaurant   
1625  4212-4214 N Milwaukee Ave          LA PENA RESTAURANTE    Restaurant   
1626       1139 W Bryn Mawr Ave         BEN'S NOODLES & RICE    Restaurant   
1627    8645 S Stony Island Ave                      WENDY'S    Restaurant   
1628        2351 W Augusta Blvd    JOEY'S G'S MAC AND CHEESE    Restaurant   

                 Risk      Zip  Year          Inspection Type R

In [14]:
inspection.to_csv('inspection_clean.csv', index=False)

In [15]:
#list all the zipcodes
zipcode = inspection['Zip'].value_counts()
zipcode

60611.0    82
60607.0    71
60622.0    67
60647.0    66
60614.0    63
60654.0    62
60638.0    55
60618.0    54
60639.0    51
60616.0    50
60628.0    49
60609.0    46
60640.0    46
60612.0    40
60657.0    37
60642.0    37
60620.0    37
60608.0    36
60610.0    35
60626.0    33
60632.0    33
60613.0    32
60617.0    32
60629.0    31
60625.0    30
60660.0    28
60601.0    28
60604.0    27
60661.0    23
60619.0    21
60659.0    21
60644.0    20
60637.0    20
60641.0    19
60630.0    18
60631.0    17
60605.0    16
60646.0    16
60649.0    16
60606.0    16
60652.0    15
60643.0    13
60615.0    12
60636.0    12
60634.0    10
60707.0    10
60666.0     9
60623.0     9
60651.0     9
60653.0     8
60645.0     7
60621.0     6
60602.0     6
60603.0     6
60827.0     5
60655.0     4
60624.0     4
60656.0     1
Name: Zip, dtype: int64

# Testing Out Merging the 2 DataFrames using lat/long